In [1]:
import os
import pandas as pd
from datetime import timedelta

In [2]:
# File paths
main_data = "C:/Data/sensors_performance/sensors_performance/"
eye_tracking_3d_state_data = "C:/Data/Eye tracking data_Main data collection/Main Data Collection/"
blinks_data_path = "C:/Data/Eye tracking data_Main data collection/Main Data Collection/"

# Prticipant code 
code = "U6O5V2T9"

Load and Align Main Data

In [3]:
# Load main data file
main_df = pd.read_csv(f'{main_data}{code}.csv', comment='#', low_memory=False)

# Extract recording timestamp from the main data file
with open(f'{main_data}{code}.csv', 'r') as f:
    for line in f:
        if line.startswith('#Recording time'):
            timestamp_str = line.replace('#Recording time,Date: ', '').split(',Unix time:')[0]
            break

# Parse the timestamp
try:
    ts = pd.to_datetime(timestamp_str, format="%d.%m.%Y,Time: %H:%M:%S.%f +02:00")
except ValueError:
    ts = pd.to_datetime(timestamp_str, format="%m/%d/%Y,Time: %H:%M:%S.%f +02:00")

# Adjust timestamp to UTC and convert to nanoseconds
ts = ts - timedelta(hours=2)
nanoseconds = int(ts.value)

Process 3D Eye States Data (including Pupil Diameter (PD))

In [4]:
# Load and align 3D eye states data
state_data = pd.read_csv(f'{eye_tracking_3d_state_data}3d_eye_states_{code}.csv', comment='#')
# Convert to milliseconds
state_data['timestamp [ns]'] = state_data['timestamp [ns]'].apply(lambda x: (x - nanoseconds) / 1e6)  
state_data.rename(columns={'timestamp [ns]': 'Timestamp'}, inplace=True)

# Save aligned 3D eye states data
output_file_3d = f'{eye_tracking_3d_state_data}aligned_3d_eye_states_{code}.csv'
state_data.to_csv(output_file_3d, index=False)
print(f"Aligned 3D eye states data saved to: {output_file_3d}")

# Merge 3D eye states with main data
merged_data = pd.merge(main_df, state_data, on='Timestamp', how='outer')

Aligned 3D eye states data saved to: C:/Data/Eye tracking data_Main data collection/Main Data Collection/aligned_3d_eye_states_U6O5V2T9.csv


Process Blink Data (including start/end and durarion time of blinks)

In [6]:
# Load and align blink data
blinks_data = pd.read_csv(f'{blinks_data_path}blinks_{code}.csv', comment='#')

# Calculate the midpoint timestamp for each blink
blinks_data['timestamp [ns]'] = (blinks_data['start timestamp [ns]'] + blinks_data['end timestamp [ns]']) // 2

# Adjust timestamps to milliseconds
blinks_data['Timestamp'] = (blinks_data['timestamp [ns]'] - nanoseconds) / 1e6
blinks_data['start timestamp [ms]'] = (blinks_data['start timestamp [ns]'] - nanoseconds) / 1e6
blinks_data['end timestamp [ms]'] = (blinks_data['end timestamp [ns]'] - nanoseconds) / 1e6

# Keep relevant columns
blinks_data = blinks_data[['Timestamp', 'start timestamp [ms]', 'end timestamp [ms]', 'duration [ms]']]

# Save aligned blink data
output_file_blinks = f'{blinks_data_path}aligned_blinks_{code}.csv'
blinks_data.to_csv(output_file_blinks, index=False)
print(f"Aligned blinks data saved to: {output_file_blinks}")


Aligned blinks data saved to: C:/Data/Eye tracking data_Main data collection/Main Data Collection/aligned_blinks_U6O5V2T9.csv


Merge All Data and Save Final Dataset

In [7]:
# Merge blink data with the existing merged dataset
merged_data = pd.merge(merged_data, blinks_data, on='Timestamp', how='outer')

# Sort data by timestamp
merged_data = merged_data.sort_values('Timestamp')

# Save the final merged dataset
final_output_file = f"C:/Data/merged_data_{code}.csv"

# Check if the file exists and delete it if necessary (for the test phase)
if os.path.exists(final_output_file):
    os.remove(final_output_file)

merged_data.to_csv(final_output_file, index=False)
print(f"Final merged data saved to: {final_output_file}")


Final merged data saved to: C:/Data/merged_data_U6O5V2T9.csv
